In [1]:
import random
import torch
import torch.nn as nn
from torch import optim


NameError: name 'torchnlp' is not defined

In [2]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

In [4]:
SOS_token = 0
EOS_token = 1

In [88]:
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}       # dictionary
        self.index2vocab = {SOS_token : "<SOS>", EOS_token: "<EOS>"}      # dictionary
        self.vocab_count = {}                                             # dictionary 단어 count
        self.n_vocab = len(self.vocab2index)                              # vocab의 index값 (temp)
        
    def add_vocab(self, sentence):
        for word in sentence.split(" "):
            if word not in self.vocab2index:                              # 만약 vocab2index dictionary에 단어가 없으면
                self.vocab2index[word] = self.n_vocab                     # vocab2index dictionary에 추가하면서 index를 표기해준다.
                self.vocab_count[word] = 1                                # 추가한 단어의 갯수 여기서는 처음 들어가는 거라 1로 고정
                self.index2vocab[self.n_vocab] = word                     # vocab2index에서 index 값에 맞게 word를 넣어준다.
                self.n_vocab += 1                                         # 인덱스 1올려준다.
            else:
                self.vocab_count[word] += 1                               # 만약 있는 단어면 근야 count만 올려준다.

In [89]:
def filter_pair(pair, source_max_length, target_max_length):              # pair에는 데이터 한줄씩 들어간다. [영어, 한글] pair[0]는 영어 pair[1]은 한글
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length

In [90]:
def preprocess(corpus, source_max_length, target_max_length):
    print('reading corpus...') 
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split('\t')])
    print('Read {} sentence pairs', format(len(pairs)))
    
    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print('Trimmed to {} sentence pairs'.format(len(pairs)))
    
    source_vocab = Vocab()
    target_vocab = Vocab()
    
    print('Counting words...')
    for pair in pairs:
        source_vocab.add_vocab(pair[0])                                   # 영어와 한글을 각각 add_vocab 해준다.
        target_vocab.add_vocab(pair[1])
    print('source vocab size = ', source_vocab.n_vocab)
    print('target vocab size = ', target_vocab.n_vocab)
    
    return pairs, source_vocab, target_vocab

In [95]:
SOURCE_MAX_LENGTH = 11
TARGET_MAX_LENGTH = 12
load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)
print(random.choice(load_pairs))

reading corpus...
Read {} sentence pairs 4
Trimmed to 4 sentence pairs
Counting words...
source vocab size =  17
target vocab size =  13
['pytorch is very clear to use.', '파이토치는 사용하기 매우 직관적이다.']


In [8]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden
    


In [9]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))
        return x, hidden
    
    
        

In [10]:
def tensorize(vocab, sentence):
    indexs = [vocab.vocab2index[word] for word in sentence.split(" ")]
    indexs.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexs).long().to(device).view(-1,1)


In [11]:
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
    loss_total = 0
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    training_batch = [random.choice(pairs) for _ in range(n_iter)]
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]
    
    criterion = nn.NLLLoss()
    
    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1]
        target_tensor = training_target[i - 1]
        
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)
        
        loss = 0
        
        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)
            
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden
        
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
            
        loss.backward()
        
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        loss_iter = loss.item() / target_length
        loss_total += loss_iter
        
        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i/n_iter * 100, loss_avg))

In [12]:
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
    for pair in pairs:
        print(">", pair[0])
        print("=", pair[1])
        source_tensor = tensorize(source_vocab, pair[0])
        source_length = source_tensor.size()[0]
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)
        
        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)
            
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden.to(device)
        decoded_words = []
        
        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1)
            if top_index.item() == EOS_token:
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])
            
            decoder_input = top_index.squeeze().detach()
            
        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        print("<", predict_sentence)
        print("")
            

reading corpus...
Read {} sentence pairs 4
Trimmed to 4 sentence pairs
Counting words...
source vocab size =  17
target vocab size =  13
['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.']


In [15]:
%%time
enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

Wall time: 2.4 s


In [16]:
train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 5000, print_every=1000)

[1000 - 20.0%] loss = 0.7506
[2000 - 40.0%] loss = 0.1767
[3000 - 60.0%] loss = 0.0531
[4000 - 80.0%] loss = 0.0229
[5000 - 100.0%] loss = 0.0144


In [17]:
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

> i feel hungry.
= 나는 배가 고프다.
< 나는 배가 고프다. <EOS>

> pytorch is very easy.
= 파이토치는 매우 쉽다.
< 파이토치는 매우 쉽다. <EOS>

> pytorch is a framework for deep learning.
= 파이토치는 딥러닝을 위한 프레임워크이다.
< 파이토치는 딥러닝을 위한 프레임워크이다. <EOS>

> pytorch is very clear to use.
= 파이토치는 사용하기 매우 직관적이다.
< 파이토치는 사용하기 매우 직관적이다. <EOS>

